# LSST Twilight Planner (Modular) — Notebook

This notebook demonstrates the use of the `twilight_planner_pkg` package to
generate twilight observing plans. The workflow is:

1. Set up imports and paths for the input SN catalog and output directory.
2. Build a fully specified `PlannerConfig` object with all optional parameters
   shown (defaults are used unless otherwise noted).
3. Run the planner and inspect the CSV outputs written to `OUTDIR`.


In [1]:

# --- Imports and path setup ---
import os
import sys
from pathlib import Path

PROJECT_ROOT = Path('..').resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from twilight_planner_pkg.config import PlannerConfig
from twilight_planner_pkg.scheduler import plan_twilight_range_with_caps

# Edit these to your paths
catalog_csv = Path('../data/ATLAS_2021_to25_with_fakes_like_input.csv')
outdir = Path("../twilight_outputs")
outdir.mkdir(parents=True, exist_ok=True)

print('Catalog CSV:', catalog_csv)
print('Catalog exists:', catalog_csv.exists())
print('Output directory:', outdir)

# Legacy aliases used by some cells below
CSV_PATH = str(catalog_csv)
OUTDIR = str(outdir)

Catalog CSV: ../data/ATLAS_2021_to25_with_fakes_like_input.csv
Catalog exists: True
Output directory: ../twilight_outputs



## Configure the planner

The next cells build a fully explicit `PlannerConfig`. A table after the configuration
lists **every available field and its current value**, so the notebook doubles as
documentation. Adjust values in the configuration cell to explore alternative
strategies or hardware assumptions. All text below is in English as requested.

### How to customise the run
- Update `catalog_csv` and `outdir` in the imports cell if your catalogue lives elsewhere.
- Edit the arguments passed to `PlannerConfig(...)` to override defaults.
- The table cell shows defaults for untouched parameters so you can audit the
  entire configuration before running the planner.

In [2]:

# ---- Planner configuration (all fields shown, edit as needed) ----
# All parameters are explicitly listed with defaults or recommended values.
# Adjust values to explore different strategies; comments explain each group.

cfg = PlannerConfig(
    # -- Site ---------------------------------------------------------------
    lat_deg=-30.2446,        # Observatory latitude (deg)
    lon_deg=-70.7494,        # Observatory longitude (deg)
    height_m=2647.0,         # Elevation above sea level (m)

    # -- Filters and hardware ---------------------------------------------
    filters=['g','r','i','z','Y'],    # Available filters (carousel)
    carousel_capacity=5,              # Carousel capacity (max filters mounted)
    filter_change_s=120.0,            # Time to change filters (s)
    readout_s=2.0,                    # CCD readout time (s)
    inter_exposure_min_s=15.0,        # Minimum time between exposures (s)
    filter_change_time_s=None,        # Legacy alias for filter_change_s
    readout_time_s=None,              # Legacy alias for readout_s
    exposure_by_filter={'u':30.0,'g':5.0,'r':5.0,'i':5.0,'z':5.0,'Y':5.0},  # Base exposure times (s)
    max_filters_per_visit=1,          # Filters allowed per target visit
    start_filter=None,                # Start filter (None → first of filters)
    sun_alt_policy=[                  # Allowed filters vs Sun altitude ranges (deg)
        (-18.0,-15.0,['Y','z','i']),
        (-15.0,-12.0,['z','i','r']),
        (-12.0,  0.0,['i','z','Y']),
    ],
    sun_alt_exposure_ladder=[],       # Optional exposure overrides per Sun-alt bucket
    filter_policy_use_best_time_alt=False,  # Use Sun alt at each target best_time (instead of window mid)

    # -- Slew model --------------------------------------------------------
    slew_small_deg=3.5,               # Small-angle definition (deg)
    slew_small_time_s=4.0,            # Time for small slew (s)
    slew_rate_deg_per_s=5.25,         # Rate for large slews (deg/s)
    slew_settle_s=1.0,                # Settling time after slew (s)

    # -- Moon --------------------------------------------------------------
    min_moon_sep_by_filter={          # Min Moon separation by filter (deg)
        'u':80.0,'g':50.0,'r':35.0,'i':30.0,'z':25.0,'Y':20.0,
    },
    require_single_time_for_all_filters=True,  # Enforce one best time for all filters of a visit (not used for now)

    # -- Time caps ---------------------------------------------------------
    min_alt_deg=30.0,                 # Min target altitude (deg)
    twilight_sun_alt_min_deg=-18.0,   # Sun altitude min for twilight window (deg)
    twilight_sun_alt_max_deg=-8.0,     # Sun altitude max for twilight window (deg)

    per_sn_cap_s=600.0,               # Max time per SN per night (s)
    morning_cap_s='auto',             # Morning twilight time cap (s or 'auto')
    evening_cap_s='auto',             # Evening twilight time cap (s or 'auto')

    # Optional manual local-time overrides for twilight windows
    morning_twilight=None,            # e.g. 'HH:MM' local start cut
    evening_twilight=None,            # e.g. 'HH:MM' local start cut

    twilight_step_min=2,              # Internal sampling step (minutes) for calculating like moon alt
    max_sn_per_night=999,    # Optional hard cap on number of SNe per night

    # -- Priority tracking -------------------------------------------------
    hybrid_detections=2,              # Detections for hybrid strategy
    hybrid_exposure_s=300.0,          # Exposure target for hybrid (s)
    lc_detections=30,                  # Detections for light-curve strategy
    lc_exposure_s=300.0,              # Exposure target for light-curve (s)
    priority_strategy='hybrid',       # 'hybrid' or 'lc' or 'unique_first'
    unique_first_fill_with_color=True,
    unique_lookback_days=999.0,       # Days before a target becomes eligible again
    unique_first_drop_threshold=0.0,  # Drop rule for very low-score targets
    unique_first_resume_score=0.0,    # Resume score after lookback window

    # -- Cadence -----------------------------------------------------------
    cadence_enable=True,              # Enable cadence gating and bonus
    cadence_per_filter=True,          # Track cadence per filter
    cadence_days_target=3.0,          # Target days between revisits
    cadence_jitter_days=0.25,         # Early revisit allowance (days)
    cadence_days_tolerance=0.5,       # KPI window around target (days)
    cadence_bonus_sigma_days=0.5,     # Width of due-soon bonus (days)
    cadence_bonus_weight=0.25,        # Weight of cadence bonus
    cadence_first_epoch_bonus_weight=0.0,  # Extra bonus for first epoch in a filter

    # -- Cosmology / colour tracking --------------------------------------
    cosmo_weight_by_filter={'g':1.25,'r':1.10,'i':1.00,'z':0.85,'Y':0.60},
    color_window_days=5.0,
    color_target_pairs=2,
    color_alpha=0.3,
    swap_cost_scale_color=0.6,
    swap_amortize_min=6,
    palette_rotation_days=4,
    palette_evening=['i','r','z','i'],
    palette_morning=['r','g','i','r'],
    max_swaps_per_window=3,
    first_epoch_color_boost=1.5,

    # -- Redshift prioritization -----------------------------------------
    redshift_boost_enable=True,       # Mild boost for low-z SNe
    redshift_low_ref=0.09,
    redshift_boost_max=5.0,
    redshift_column=None,             # Explicit redshift column in catalog (optional)

    # -- Low-z Ia special handling ----------------------------------------
    low_z_ia_markers=['ia','1','101'],
    low_z_ia_z_threshold=0.05,
    low_z_ia_priority_multiplier=1.0,
    low_z_ia_cadence_days_target=None,
    low_z_ia_repeats_per_window=None,

    # -- Backfill relax cadence -------------------------------------------
    backfill_relax_cadence=True,     # Last-resort: relax cadence if time would go unused

    # -- Catalog pre-filtering --------------------------------------------
    only_ia=True,                    # Pre-filter to Ia-like types only

    # -- Photometry / sky --------------------------------------------------
    pixel_scale_arcsec=0.2,           # Arcsec per pixel
    zpt1s=None,                       # Optional per-filter zeropoint@1s
    k_m=None,                         # Optional per-filter extinction coefficient
    fwhm_eff=None,                    # Optional per-filter effective FWHM
    read_noise_e=6.0,                 # Read noise (e-)
    gain_e_per_adu=1.6,               # Gain (e-/ADU)
    zpt_err_mag=0.01,                 # Zeropoint error (mag)
    dark_sky_mag=None,                # Optional per-filter dark-sky mags
    twilight_delta_mag=2.5,           # Brightening of sky in twilight if no provider

    # -- Cosmology / peak-magnitude guardrails ----------------------------
    H0_km_s_Mpc=70.0,
    Omega_m=0.3,
    Omega_L=0.7,
    MB_absolute=-19.36,
    SALT2_alpha=0.14,
    SALT2_beta=3.1,
    Kcorr_approx_mag=0.0,
    Kcorr_approx_mag_by_filter={},
    peak_extra_bright_margin_mag=0.3, # Extra margin when using peak mags (mag)

    # -- SIMLIB ------------------------------------------------------------
    simlib_out=None,                  # Will be set below to write a SIMLIB file
    simlib_survey='LSST',
    simlib_filters='grizY',           # SNANA filter string (we may set 'ugrizY')
    simlib_pixsize=0.2,
    simlib_npe_pixel_saturate=80000.0,
    simlib_photflag_saturate=2048,
    simlib_psf_unit='PIXEL',          # SNANA expects PSF1/PSF2/PSFRATIO in PIXEL

    # -- Miscellaneous -----------------------------------------------------
    typical_days_by_type={'Ia':70,'II-P':100,'II-L':70,'IIn':120,'IIb':70,'Ib':60,'Ic':60},
    default_typical_days=60,
    ra_col=None,
    dec_col=None,
    disc_col=None,
    name_col=None,
    type_col=None,

    # -- Scheduler-populated context (leave None) -------------------------
    current_mag_by_filter=None,
    current_alt_deg=None,
    current_mjd=None,
    current_redshift=None,
    sky_provider=None,

    # -- Optional per-target host galaxy context --------------------------
    current_host_mu_arcsec2_by_filter=None,
    current_host_mu_rest_arcsec2_by_filter=None,
    current_host_z=None,
    current_host_K_by_filter=None,
    current_host_point_mag_by_filter=None,
    current_host_point_frac=None,

    # -- Default host SB fallback when per-target inputs are missing ------
    use_default_host_sb=True,
    default_host_mu_arcsec2_by_filter={'u':22.0,'g':22.0,'r':22.0,'i':22.0,'z':22.0,'Y':22.0},
    default_host_mu_rest_arcsec2_by_filter={'u':22.8,'g':22.2,'r':21.7,'i':21.5,'z':21.4,'Y':21.3},
    default_host_kcorr_slope_by_filter={'u':0.35,'g':0.30,'r':0.25,'i':0.20,'z':0.20,'Y':0.15},

    # -- Backwards-compatibility options ----------------------------------
    allow_filter_changes_in_twilight=True,  # Permit changing filters within twilight

    # -- Discovery magnitude fallback (for saturation capping) -------------
    use_discovery_fallback=True,
    discovery_policy='atlas_priors',
    discovery_assumed_gr=0.0,
    discovery_margin_mag=0.2,
    discovery_color_priors_min={'u-g':0.3,'g-r':-0.25,'r-i':-0.15,'i-z':-0.10,'z-y':-0.05},
    discovery_color_priors_max={'u-g':1.0,'g-r':0.15,'r-i':0.25,'i-z':0.20,'z-y':0.30},
    discovery_non_ia_widen_mag=0.1,
    discovery_y_extra_margin_mag=0.25,
    discovery_atlas_linear={'c':{'alpha':0.0,'beta':-0.47}, 'o':{'alpha':0.0,'beta':0.26}},
    discovery_error_on_missing=True,
)

# Recommended SIMLIB header consistency for SNANA
cfg.simlib_filters = 'grizY'  # include 'Y' if needed for upstream tools
cfg.simlib_pixsize = 0.200
cfg.simlib_psf_unit = 'PIXEL'

In [3]:

import dataclasses
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

config_rows = []
for field in dataclasses.fields(PlannerConfig):
    value = getattr(cfg, field.name)
    config_rows.append({'field': field.name, 'type': str(field.type), 'value': repr(value)})

cfg_overview = pd.DataFrame(config_rows)
display(cfg_overview)

,field,type,value
0,lat_deg,float,-30.2446
1,lon_deg,float,-70.7494
2,height_m,float,2647.0
3,min_alt_deg,float,30.0
4,twilight_sun_alt_min_deg,float,-18.0
5,twilight_sun_alt_max_deg,float,-8.0
6,filters,List[str],"['g', 'r', 'i', 'z', 'Y']"
7,carousel_capacity,int,5
8,filter_change_s,float,120.0
9,readout_s,float,2.0


In [4]:
# Use SNANA central-pixel fraction (Taylor expansion) for parity
from twilight_planner_pkg.photom_rubin import PhotomConfig
phot_defaults = PhotomConfig(
    pixel_scale_arcsec=cfg.pixel_scale_arcsec,
    gain_e_per_adu=cfg.gain_e_per_adu,
    npe_pixel_saturate=cfg.simlib_npe_pixel_saturate,
    npe_pixel_warn_nonlinear=int(0.8 * cfg.simlib_npe_pixel_saturate),
    use_snana_fA=True
)


In [5]:
from functools import lru_cache
from twilight_planner_pkg import astro_utils as AU

@lru_cache(maxsize=4096)
def _peak_mag_cached(z: float, band: str) -> float:
    return AU.peak_mag_from_redshift(
        float(z), str(band).lower(),
        MB=cfg.MB_absolute, alpha=cfg.SALT2_alpha, beta=cfg.SALT2_beta,
        H0=cfg.H0_km_s_Mpc, Om=cfg.Omega_m, Ol=cfg.Omega_L,
        K_approx=cfg.Kcorr_approx_mag_by_filter.get(str(band).lower(), 0.0),
    )

# Route internal calls through the cache
AU.peak_mag_from_redshift = _peak_mag_cached


In [6]:
# Diagnose catalog magnitude mapping for discovery fallback
from twilight_planner_pkg.examine import diagnose_mag_mapping
band_to_col, maglike_cols, preview = diagnose_mag_mapping(str(catalog_csv), cfg)
print('Band→column mapping:', band_to_col)
print('Columns containing "mag":', maglike_cols[:10])
display(preview)



RA raw (numeric) range:  min=0.043167, max=359.855458
Dec raw (numeric) range: min=-87.141200, max=87.960305
Band→column mapping: {'g': None, 'r': None, 'i': None, 'z': None, 'y': None}
Columns containing "mag": ['discoverymag', 'discmagfilter']


,objid,name_prefix,name,ra,declination,redshift,typeid,type,reporting_groupid,reporting_group,...,Discovery_ADS_bibcode,Class_ADS_bibcodes,creationdate,lastmodified,RA_deg,Dec_deg,discovery_datetime,Name,SN_type_raw,typical_lifetime_days
0,185080,SN,2025som,51.901204,-3.772340,0.025992,3.0,SN Ia,18.0,ATLAS,...,2025TNSTR2966....1T,2025TNSCR3090....1Y,2025-07-29 13:19:18,2025-08-05 19:48:11,51.901204,-3.772340,2025-07-29 03:46:34.176000+00:00,2025som,SN Ia,84
1,184289,SN,2025rlk,2.922341,-22.986027,0.064400,3.0,SN Ia,18.0,ATLAS,...,2025TNSTR2748....1T,2025TNSCR3056....1D,2025-07-18 23:50:26,2025-08-03 15:27:23,2.922341,-22.986027,2025-07-16 03:31:34.752000+00:00,2025rlk,SN Ia,84
2,185029,SN,2025smn,333.906980,-30.583877,0.086000,3.0,SN Ia,18.0,ATLAS,...,2025TNSTR2947....1T,2025TNSCR3047....1F,2025-07-28 09:55:05,2025-08-02 13:49:05,333.906980,-30.583877,2025-07-26 05:05:01.248000+00:00,2025smn,SN Ia,84
3,173276,SN,2025cln,38.600728,16.918474,0.027820,3.0,SN Ia,18.0,ATLAS,...,2025TNSTR.734....1T,2025TNSCR3019....1D,2025-02-23 10:51:44,2025-07-31 22:51:34,38.600728,16.918474,2025-02-23 05:48:08.064000+00:00,2025cln,SN Ia,84
4,183800,SN,2025quo,195.447897,1.493214,0.070000,3.0,SN Ia,18.0,ATLAS,...,2025TNSTR2646....1T,2025TNSCR2880....1S,2025-07-11 12:39:59,2025-07-30 06:01:07,195.447897,1.493214,2025-07-10 06:35:22.848000+00:00,2025quo,SN Ia,84


## Run the planner

`plan_twilight_range_with_caps` is invoked below.  It reads the SN catalog from
`catalog_csv` (aliased as `CSV_PATH` for legacy cells), writes output CSVs to `outdir`
/ `OUTDIR`, and prints progress information.
Two files are produced for each run:

- `lsst_twilight_plan_<run_label>_*.csv` — per-SN schedule
- `lsst_twilight_summary_<run_label>_*.csv` — per-night summary


In [7]:

start_date = '2024-01-01'
end_date = '2024-01-03'

# Mirror legacy variable names for later cells
START_DATE = start_date
END_DATE = end_date

# Enable SIMLIB output (SNANA expects PSF_UNIT=PIXEL)
cfg.simlib_out = str((outdir / 'lsst_twilight.simlib').resolve())

perSN_df, nights_df = plan_twilight_range_with_caps(
    csv_path=str(catalog_csv),
    outdir=str(outdir),
    start_date=start_date,
    end_date=end_date,
    cfg=cfg,
    run_label='hybrid',
    verbose=True
)

RA raw (numeric) range:  min=0.043167, max=359.855458
Dec raw (numeric) range: min=-87.141200, max=87.960305


Nights:   0%|          | 0/3 [00:00<?, ?night/s]

2024-01-01: eligible=619 visible=374 planned_total=283
  evening_twilight: 2024-01-02T00:27:00+00:00 → 2024-01-02T01:24:00+00:00 (local 19:44 → 20:41 UTC-04:43)
  morning_twilight: 2024-01-02T08:09:00+00:00 → 2024-01-02T09:06:00+00:00 (local 03:26 → 04:23 UTC-04:43)
2024-01-02: eligible=642 visible=401 planned_total=283
  evening_twilight: 2024-01-03T00:27:00+00:00 → 2024-01-03T01:24:00+00:00 (local 19:44 → 20:41 UTC-04:43)
  morning_twilight: 2024-01-03T08:10:00+00:00 → 2024-01-03T09:07:00+00:00 (local 03:27 → 04:24 UTC-04:43)
2024-01-03: eligible=646 visible=413 planned_total=271
  evening_twilight: 2024-01-04T00:27:00+00:00 → 2024-01-04T01:24:00+00:00 (local 19:44 → 20:41 UTC-04:43)
  morning_twilight: 2024-01-04T08:11:00+00:00 → 2024-01-04T09:08:00+00:00 (local 03:28 → 04:25 UTC-04:43)
Wrote:
  ../twilight_outputs/lsst_twilight_plan_hybrid_2024-01-01T00:00:00+00:00_to_2024-01-03T00:00:00+00:00.csv
  ../twilight_outputs/lsst_twilight_summary_hybrid_2024-01-01T00:00:00+00:00_to_2024-

In [8]:
perSN_df.head()

,date,twilight_window,SN,RA_deg,Dec_deg,best_twilight_time_utc,visit_start_utc,sn_end_utc,filter,t_exp_s,...,cadence_gate_passed,slew_s,cross_filter_change_s,filter_changes_s,readout_s,exposure_s,guard_s,inter_exposure_guard_enforced,total_time_s,elapsed_overhead_s
0,2024-01-01,evening,2023xbc,18.756059,-33.018606,2024-01-02T00:27:00+00:00,2024-01-02T00:27:00+00:00,2024-01-02T00:31:07+00:00,r,5.0,...,True,0.0,120.0,120.0,2.0,5.0,0.0,False,247.0,242.0
1,2024-01-01,evening,2023zrc,17.044547,-36.059907,2024-01-02T00:27:00+00:00,2024-01-02T00:31:07+00:00,2024-01-02T00:31:27+00:00,r,5.0,...,True,5.0,0.0,0.0,2.0,5.0,10.0,True,20.0,5.0
2,2024-01-01,evening,23fklmt,17.044547,-36.059907,2024-01-02T00:27:00+00:00,2024-01-02T00:31:27+00:00,2024-01-02T00:31:47+00:00,r,5.0,...,True,0.0,0.0,0.0,2.0,5.0,13.0,True,20.0,2.0
3,2024-01-01,evening,23fkrfy,17.044547,-36.059907,2024-01-02T00:27:00+00:00,2024-01-02T00:31:47+00:00,2024-01-02T00:32:07+00:00,r,5.0,...,True,0.0,0.0,0.0,2.0,5.0,13.0,True,20.0,2.0
4,2024-01-01,evening,23fkrsx,17.044547,-36.059907,2024-01-02T00:27:00+00:00,2024-01-02T00:32:07+00:00,2024-01-02T00:32:27+00:00,r,5.0,...,True,0.0,0.0,0.0,2.0,5.0,13.0,True,20.0,2.0


In [9]:
# Show the nightly summary
nights_df

,date,twilight_window,n_candidates,n_planned,unique_targets_observed,repeat_fraction,sum_time_s,window_cap_s,swap_count,internal_filter_changes,...,cap_source,median_sky_mag_arcsec2,median_alt_deg,cad_median_abs_err_by_filter_csv,cad_within_pct_by_filter_csv,cad_median_abs_err_all_d,cad_within_pct_all,quota_assigned,n_candidates_pre_cap,n_candidates_post_cap
0,2024-01-01,evening,253,159,159,0.000,3407.0,3420,1,0,...,window_duration,19.831389,62.43,,,NaN,NaN,500,253,253
1,2024-01-01,morning,121,124,91,0.266,3417.9,3420,4,0,...,window_duration,19.746779,54.31,,,NaN,NaN,499,121,121
2,2024-01-02,evening,251,148,148,0.000,3418.2,3420,2,0,...,window_duration,19.729830,58.91,,,NaN,NaN,500,251,251
3,2024-01-02,morning,150,135,115,0.148,3406.4,3420,3,0,...,window_duration,19.031910,48.69,,,NaN,NaN,499,150,150
4,2024-01-03,evening,252,148,148,0.000,3417.8,3420,2,0,...,window_duration,19.822108,64.21,,,NaN,NaN,500,252,252
5,2024-01-03,morning,161,123,99,0.195,3402.8,3420,4,0,...,window_duration,19.646565,39.78,,,NaN,NaN,499,161,161



## Inspect outputs on disk


In [10]:
import glob
import os

files = sorted(glob.glob(os.path.join(OUTDIR, "lsst_twilight_*.csv")))
files

['../twilight_outputs/lsst_twilight_plan_hybrid_2023-01-01T00:00:00+00:00_to_2023-01-05T00:00:00+00:00.csv',
 '../twilight_outputs/lsst_twilight_plan_hybrid_2023-01-01T00:00:00+00:00_to_2023-01-20T00:00:00+00:00.csv',
 '../twilight_outputs/lsst_twilight_plan_hybrid_2023-01-01T00:00:00+00:00_to_2023-03-01T00:00:00+00:00.csv',
 '../twilight_outputs/lsst_twilight_plan_hybrid_2023-01-01T00:00:00+00:00_to_2024-01-01T00:00:00+00:00.csv',
 '../twilight_outputs/lsst_twilight_plan_hybrid_2024-01-01T00:00:00+00:00_to_2024-01-03T00:00:00+00:00.csv',
 '../twilight_outputs/lsst_twilight_sequence_true_hybrid_2023-01-01T00:00:00+00:00_to_2023-01-05T00:00:00+00:00.csv',
 '../twilight_outputs/lsst_twilight_sequence_true_hybrid_2023-01-01T00:00:00+00:00_to_2023-01-20T00:00:00+00:00.csv',
 '../twilight_outputs/lsst_twilight_sequence_true_hybrid_2023-01-01T00:00:00+00:00_to_2023-03-01T00:00:00+00:00.csv',
 '../twilight_outputs/lsst_twilight_sequence_true_hybrid_2023-01-01T00:00:00+00:00_to_2024-01-01T00:0

In [11]:
# Diagnose per-visit saturation contributions
from twilight_planner_pkg.examine import build_saturation_df
saturation_df = build_saturation_df(perSN_df, CSV_PATH, cfg)
# Save and show a compact view
sat_path = os.path.join(OUTDIR, f'saturation_{START_DATE}_to_{END_DATE}.csv')
saturation_df.to_csv(sat_path, index=False)
cols = ['SN','filter','t_exp_s','t_exp_s_base','src_mag','sky_mag_arcsec2','host_mu_arcsec2',
        'e_src','e_sky','e_host','total_e','sat_limit','sat_guard_applied','warn_nonlinear',
        'over_sat','over_sat_base']
display(saturation_df[cols].head(20))
print('Wrote:', sat_path)



RA raw (numeric) range:  min=0.043167, max=359.855458
Dec raw (numeric) range: min=-87.141200, max=87.960305


,SN,filter,t_exp_s,t_exp_s_base,src_mag,sky_mag_arcsec2,host_mu_arcsec2,e_src,e_sky,e_host,total_e,sat_limit,sat_guard_applied,warn_nonlinear,over_sat,over_sat_base
0,2023xbc,r,5.0,5.0,15.247156,18.23,21.800009,43741.063622,2245.846005,83.825546,46070.735174,80000.0,False,False,-33929.264826,-33915.612742
1,2023zrc,r,5.0,5.0,18.835000,18.45,22.222180,1604.543751,1832.229933,56.768561,3493.542246,80000.0,False,False,-76506.457754,-76505.477461
2,23fklmt,r,5.0,5.0,18.835000,18.47,22.180730,1604.543751,1798.787968,58.977731,3462.309450,80000.0,False,False,-76537.690550,-76536.719020
3,23fkrfy,r,5.0,5.0,18.835000,18.49,22.180730,1604.543751,1765.956388,58.977731,3429.477870,80000.0,False,False,-76570.522130,-76569.559813
4,23fkrsx,r,5.0,5.0,18.835000,18.51,22.197352,1604.543751,1733.724051,58.081685,3396.349488,80000.0,False,False,-76603.650512,-76602.697491
5,23fkiwa,r,5.0,5.0,18.835000,18.53,22.263284,1604.543751,1702.080021,54.659569,3361.283341,80000.0,False,False,-76638.716659,-76637.773477
6,23fkped,r,5.0,5.0,18.835000,18.54,22.222180,1604.543751,1686.475258,56.768561,3347.787570,80000.0,False,False,-76652.212430,-76651.273036
7,23fkyjq,r,5.0,5.0,18.835000,18.56,22.193202,1604.543751,1655.693615,58.304130,3318.541496,80000.0,False,False,-76681.458504,-76680.527316
8,23fkuls,r,5.0,5.0,18.768406,18.58,22.159873,1704.468907,1623.977373,60.066297,3388.512578,80000.0,False,False,-76611.487422,-76609.934216
9,23fktpg,r,5.0,5.0,18.734990,18.60,22.153598,1757.743282,1594.336446,60.414422,3412.494150,80000.0,False,False,-76587.505850,-76585.941651


Wrote: ../twilight_outputs/saturation_2024-01-01_to_2024-01-03.csv


In [12]:
print("Filters used:", cfg.filters)
print("SIMLIB sat (NPE_PIXEL_SATURATE):", cfg.simlib_npe_pixel_saturate)
print("PSF_UNIT:", cfg.simlib_psf_unit)

# If SIMLIB was written, glance at header lines
if cfg.simlib_out:
    with open(cfg.simlib_out, "r") as fp:
        for i, line in enumerate(fp):
            if i > 25:
                break
            print(line.rstrip())


Filters used: ['g', 'r', 'i', 'z', 'Y']
SIMLIB sat (NPE_PIXEL_SATURATE): 80000.0
PSF_UNIT: PIXEL
DOCUMENTATION:
  PURPOSE: LSST twilight cadence library for SNANA simulation
  INTENT: Nominal
  USAGE_KEY: SIMLIB_FILE
  NOTE: SKYSIG and NOISE columns are ADU/pixel; SKYSIG excludes read noise.
  NOTE: RDNOISE is converted via gain so SNANA recovers electrons per pixel.
DOCUMENTATION_END:

# Generated by twilight_planner
SURVEY:   LSST
FILTERS:  grizY
PIXSIZE:  0.200
NPE_PIXEL_SATURATE: 80000.0
PHOTFLAG_SATURATE:  2048
PSF_UNIT:  PIXEL

BEGIN LIBGEN

#--------------------------------------------
LIBID:       1     # 2023xbc
NOBS: 2  RA:   18.756059  DEC:  -33.018606
REDSHIFT:  0.02200     PEAKMJD:   60312.026

#     MJD        ID+NEXPOSE FLT GAIN NOISE SKYSIG PSF1 PSF2 PSFRATIO ZPTAVG ZPTERR
S:  60311.0188            1  r   1.60   3.75    29.61    1.762    0.000    0.000  29.593  0.010
S:  60313.0332            2  i   1.60   3.75    14.95    1.762    0.000    0.000  29.404  0.010
